# Part I - Explanation of the problem and why it is interesting

## 1 - Problem

Indication: Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

Israel is a very specific place in the arabic world with very specific needs and demand. Since it is creation on the 14th of May 1948 after the vote by the ONU, it has been largely influenced by the international community and specifically the occidental culture.
Nevertheless, Isreal remains part of the Arabic world and is affected by its culture.

This combination of cultures makes the understanding of israely's culture and its consumption behaviour extremely difficult.

Several international companies would like to understand better this context to establish a strategy plan for implementation.

#### Nota bene: there is nothing politic about this notebook its purpose is purely scientific.

## 2 - Data

Indication: Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

We would like to benchmark Israel's capital Tel Aviv versus two well known cities to establish the consumption behaviour of its population.

For the benchmark, we have selected New York City and Amman for which we will be using Four Square's location data and compare the venues of the three cities in terms of shops, cafes, restaurant and so on. By establishing the profile in terms of venues in Four square we should be able to caracterise Tel Aviv and then compare it to AMman and New York City. We will then investigate the differences and conclude by identifying the best proxy.

Indeed, the number of venues will be divided by the population of each city to remain consistent.

If it happens that none of the two cities are close enough, we will iterate with other cities among the top 50 largest cities.

# Part II - Method to solve it

## 0 - Import of libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
CLIENT_ID = 'DCPMWEGCH0C2PZR13QN0IB0KEJSDLZMIPARV415TX5Y52FQ1' # your Foursquare ID
CLIENT_SECRET = 'W3BY4ARSZPO4YLRLD4BHCEWHJKLZW5UJYHEHEVDPKVXGRZXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DCPMWEGCH0C2PZR13QN0IB0KEJSDLZMIPARV415TX5Y52FQ1
CLIENT_SECRET:W3BY4ARSZPO4YLRLD4BHCEWHJKLZW5UJYHEHEVDPKVXGRZXX


In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 1 - Exploration: Amman

In [7]:
address = 'Amman'

geolocator = Nominatim(user_agent="my access")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amman are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amman are 31.9515694, 35.9239625.


In [8]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius in meters

# create URL

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fakhr Al Din (مطعم فخر الدين),Middle Eastern Restaurant,31.952205,35.920381
1,Jo Bedu (جوبدو),Clothing Store,31.956443,35.926776
2,Mijana (ميجنا),Café,31.950485,35.925520
3,Rumi Cafe (مقهى رومي),Café,31.956113,35.925881
4,Falafel Al-Quds (فلافل القدس),Falafel Restaurant,31.949464,35.926499


In [10]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [11]:
venues_grouped = nearby_venues.drop(['lat', 'lng'], axis=1).groupby('categories').count().sort_values(['name'], ascending=[False]).reset_index()
venues_grouped.rename(columns={'name':'count'}, inplace=True)
venues_grouped.head(10)

,categories,count
0,Café,12
1,Hotel,8
2,Coffee Shop,6
3,Middle Eastern Restaurant,5
4,Italian Restaurant,5
5,Ice Cream Shop,4
6,Dessert Shop,4
7,Pub,3
8,Historic Site,3
9,Bakery,3


# 2 - Deploy the same analysis on other cities

In [47]:
# function that extracts the category of the venue
def collect_venues(address):
    geolocator = Nominatim(user_agent="my access")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    LIMIT = 10000 # limit of number of venues returned by Foursquare API

    radius = 20000 # define radius in meters

    # create URL

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
    results = requests.get(url).json()

    venues = results['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.location.city', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    # one hot encoding
    result_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    #result_onehot['Name'] = nearby_venues['name']
    result_onehot['City'] = address

    # move neighborhood column to the first column
    fixed_columns = [result_onehot.columns[-1]] + list(result_onehot.columns[:-1])
    result_onehot = result_onehot[fixed_columns]
    
    result_onehot = result_onehot.groupby('City').mean().reset_index()    

    #venues_grouped = nearby_venues.drop(['lat', 'lng'], axis=1).groupby('categories').count().reset_index()
    #venues_grouped.rename(columns={'name':'count'}, inplace=True)
    return result_onehot

In [ ]:
#Collect a dataframe with the 50 largest cities with


In [99]:
column_name = list()

result = collect_venues('Amman')
column_name.extend(list(result.columns))

result = collect_venues('New York City')
column_name.extend(list(result.columns))

result = collect_venues('Tel Aviv')
column_name.extend(list(result.columns))

column_name = sorted(column_name)
column_name = list(dict.fromkeys(column_name))

global_df = pd.DataFrame(columns=column_name)
global_df = global_df.append(collect_venues('Amman'), sort=False)
global_df = global_df.append(collect_venues('New York City'), sort=False)
global_df = global_df.append(collect_venues('Tel Aviv'), sort=False)

global_df.fillna(0, inplace=True)
global_df.set_index('City', inplace=True)
global_df

,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridge,Building,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Juice Bar,Market,Mediterranean Restaurant,Memorial Site,Middle Eastern Restaurant,Movie Theater,Multiplex,Music Venue,Neighborhood,Opera House,Park,Pedestrian Plaza,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Amman,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.03,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.02,0.11,0.00,0.02,0.00,0.01,0.00,0.01,0.06,0.00,0.00,0.00,0.00,0.00,0.06,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.00,0.02,0.01,0.03,0.00,0.01,0.08,0.00,0.04,0.00,0.00,0.04,0.02,0.00,0.01,0.00,0.05,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.02,0.01,0.02,0.01,0.00,0.00,0.01,0.00,0.00,0.02,0.01,0.00,0.01,0.00,0.02,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00
New York City,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.04,0.01,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.00,0.01,0.01,0.03,0.02,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.00,0.02,0.02,0.00,0.00,0.01,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.01,0.05,0.01,0.00,0.03,0.01,0.01,0.02,0.01,0.00,0.02,0.00,0.01,0.00,0.00,0.13,0.00,0.02,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.02,0.01,0.01,0.04,0.01,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.02,0.00,0.02
Tel Aviv,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.06,0.05,0.08,0.01,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.02,0.08,0.00,0.00,0.00,0.00,0.00,0.04,0.05,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.06,0.00,0.07,0.00,0.03,0.01,0.01,0.01,0.03,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.00,0.00,0.03,0.01,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.00,0.01,0.01,0.01,0.02,0.00,0.00,0.00,0.00,0.00


In [54]:
result_nyc = collect_venues('new york city')
result_nyc

,City,American Restaurant,Art Gallery,Athletics & Sports,Bagel Shop,Bakery,Beach,Beer Store,Bookstore,Boutique,Bridge,Building,Café,Cheese Shop,Chocolate Shop,Club House,Cocktail Bar,Comedy Club,Cosmetics Shop,Cycle Studio,Dance Studio,Farm,Farmers Market,Fish Market,Flower Shop,Furniture / Home Store,Garden,Greek Restaurant,Grocery Store,Gym,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Juice Bar,Market,Mediterranean Restaurant,Memorial Site,Movie Theater,Music Venue,Park,Pier,Pilates Studio,Playground,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Soccer Field,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Trail,Volleyball Court,Wine Bar,Wine Shop,Yoga Studio
0,new york city,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.05,0.01,0.03,0.01,0.01,0.02,0.01,0.02,0.01,0.13,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02


In [66]:
result_telaviv = collect_venues('tel aviv')
result_telaviv

,categories,count
0,Art Museum,1
1,Asian Restaurant,2
2,Bakery,6
3,Bar,5
4,Beach,8
5,Bed & Breakfast,1
6,Beer Bar,1
7,Bistro,1
8,Bookstore,1
9,Burger Joint,2


In [67]:
result = result_amman + result_nyc
result

,categories,count
0,American RestaurantAmerican Restaurant,2.0
1,Art GalleryArt Gallery,3.0
2,Arts & Crafts StoreAthletics & Sports,2.0
3,BakeryBagel Shop,4.0
4,BarBakery,2.0
5,BoutiqueBeach,2.0
6,Breakfast SpotBeer Store,3.0
7,Burger JointBookstore,6.0
8,CaféBoutique,12.0
9,Chinese RestaurantBridge,3.0
